# Import Data

* Note that the dataframe "df" is being reused and over-written for memory saving purpose.
* Description of data schemas is in https://github.com/Azure/AzurePublicDataset/blob/master/AzureFunctionsDataset2019.md
* The original datasets are in CSVs

In [1]:
# importing necesary libraries
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import round, col

StatementMeta(genterist, 2, 1, Finished, Available)

In [2]:
%%pyspark
df = spark.read.load([
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d01.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d02.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d03.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d04.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d05.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d06.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d07.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d08.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d09.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d10.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d11.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d12.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d13.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/invocations_per_function_md.anon.d14.csv'
    ], format='csv'
, header=True
)
df = df.select("HashOwner","HashApp","HashFunction","Trigger")
df = df.distinct()
## get only http triggers
df = df.where("Trigger == 'http'")
df.summary().show()
#display(df.limit(10))
df.write.mode("overwrite").saveAsTable("triggers")

## importing function durations
df = spark.read.load([
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d01.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d02.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d03.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d04.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d05.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d06.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d07.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d08.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d09.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d10.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d11.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d12.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d13.csv',
    'abfss://genfilesys@gensynapse.dfs.core.windows.net/Azure-functionDataset-2019/function_durations_percentiles.anon.d14.csv'
    ], format='csv'
, header=True
)
## do not need to save to a new table here since we'll use temp tables later on

StatementMeta(genterist, 2, 2, Finished, Available)

+-------+--------------------+--------------------+--------------------+-------+
|summary|           HashOwner|             HashApp|        HashFunction|Trigger|
+-------+--------------------+--------------------+--------------------+-------+
|  count|               30440|               30440|               30440|  30440|
|   mean|                null|                null|                null|   null|
| stddev|                null|                null|                null|   null|
|    min|0001edd756889b6f8...|000143d27de4ac74f...|000048406e45e3cdf...|   http|
|    25%|                null|                null|                null|   null|
|    50%|                null|                null|                null|   null|
|    75%|                null|                null|                null|   null|
|    max|fffce5f6d8522e7ca...|fffce52236bf5616a...|fffbedc79ab93024e...|   http|
+-------+--------------------+--------------------+--------------------+-------+



# Transform data

In [3]:
%%pyspark
## minimum occurances of a tuple, used to filter number of http trigger
## there is no need for including http trigger functions with less than n occurences because as the time-series models learn how to predict
## occurences later than the n(th), they already learned how to predict the (n-x) occurances.
occurances = 9

## log the original order of functions by Func_index
df = df.withColumn('Func_index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

## log the original order of each function's occurences by sub_index
my_window = Window.partitionBy("HashFunction").orderBy("HashOwner","HashApp","HashFunction")  # very important to note we set a table's partition here
df = df.withColumn('sub_index', row_number().over(my_window))
df = df.orderBy("HashOwner","HashApp","HashFunction","sub_index")
df = df.select("HashOwner","HashApp","HashFunction","Func_index","Sub_index","Average","Minimum","Maximum")
print("Original length: ",df.count())

## Grab the tuples with more than occurances
df2 = df.select("HashOwner","HashApp","HashFunction","Sub_index")
df2 = df2.where(df2.Sub_index>occurances)
print("Tuples with more than ",occurances," occurances: ",df2.count())
df2 = df2.select("HashOwner","HashApp","HashFunction")
df2 = df2.distinct()
print("DISTINCT tuples with more than ",occurances," occurances:",df2.count())

## Filter the main df with targeted tuples
df2.createOrReplaceTempView("df2")
df2 = spark.sql("SELECT * FROM df2 NATURAL JOIN triggers")
print("DISTINCT tuples relating to http triggers with more than ",occurances," occurances:",df2.count())
df2.createOrReplaceTempView("df2")
df.createOrReplaceTempView("df")
df = spark.sql("SELECT * FROM df NATURAL JOIN df2")
print("Original length filtered: ",df.count())

## order by original Index
df = df.orderBy("Func_index", ascending=True)


## generate final Index
# w = Window.orderBy("Func_index")
# df = df.withColumn("index", row_number().over(w)-1)

display(df.limit(5))

StatementMeta(genterist, 2, 3, Finished, Available)

Original length:  662927
Tuples with more than  9  occurances:  166910
DISTINCT tuples with more than  9  occurances: 38750
DISTINCT tuples relating to http triggers with more than  9  occurances: 10636
Original length filtered:  136997


SynapseWidget(Synapse.DataFrame, 434a0dbc-1c20-45c9-a732-970b6b86e74e)

In [4]:
%%pyspark
df = df.select("Func_index","HashOwner","HashApp","HashFunction","Average","Minimum","Maximum")
## print out some basic stats
# df.summary().show()
## print variance of func_index
df.agg({'Func_index': 'variance'}).show()
display(df.limit(10))

StatementMeta(genterist, 2, 4, Finished, Available)

+--------------------+
|variance(Func_index)|
+--------------------+
|3.670791866628761E10|
+--------------------+



SynapseWidget(Synapse.DataFrame, 6a8ae44e-6b95-4ef1-bfe1-1a50b83f83ee)

In [5]:
from pyspark.ml.feature import StringIndexer
function_indexer = StringIndexer(inputCol="HashFunction", outputCol="Func_ID")
df_new = function_indexer.fit(df).transform(df)

StatementMeta(genterist, 2, 5, Finished, Available)

In [6]:
## Visual checks
df_new.orderBy("Func_index", ascending=False).show(10)
df_new.orderBy("Func_index", ascending=True).show(10)
df_new.count()

StatementMeta(genterist, 2, 6, Finished, Available)

+----------+--------------------+--------------------+--------------------+-------+-------+-------+-------+
|Func_index|           HashOwner|             HashApp|        HashFunction|Average|Minimum|Maximum|Func_ID|
+----------+--------------------+--------------------+--------------------+-------+-------+-------+-------+
|    662922|9ba133aa96ee06d21...|ac032fd6a52508d2d...|cd296f64ab757c661...|   5927| 5927.0| 5927.0| 5144.0|
|    662914|be54c9e3ff09f861f...|d8218a33de5b12acb...|48145a2f10ca4e085...|  48137|48137.0|48137.0| 6611.0|
|    662911|0e6f4c0d10481d3e2...|96472968a363a279f...|8141d8d366d53cecc...|  65661|65661.0|65661.0| 3251.0|
|    662907|c4ed082598a4332e5...|3d5835bab5a20f59b...|34b11f4fe1ffee918...|   2405| 2405.0| 2405.0| 1349.0|
|    662898|7369f94603b35208b...|8ba068bdd814802c2...|48312d98155341be6...|     46|   38.0|   53.0| 1856.0|
|    662896|d8a13b0cad5805789...|c84dbd26827cc16d9...|8fd372dd4bb570578...|     87|    0.0|  234.0| 3592.0|
|    662895|653935f489b6ad9c

136997

In [7]:
df_new = df_new.select("Func_index","HashOwner","Func_ID","Average","Minimum","Maximum")
df_new = df_new.orderBy("Func_index", ascending=True)
## Split df to train and test set
test_size = 500 # equal or greater than forcast horizon
train_size = df_new.count() - test_size
df_train = df_new.limit(train_size)
df_new = df_new.orderBy("Func_index", ascending=False) #flip the table
df_test = df_new.limit(test_size)
df_test = df_test.orderBy("Func_index", ascending=True) #flip df_test back, don't forget to flip df back if you want to use it later on
display(df_train.limit(10))
df_test.tail(10)

StatementMeta(genterist, 2, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 86384087-67d9-40ba-9b52-86835c9e8575)

[Row(Func_index=662886, HashOwner='b0eff6ce3f584d3aceca3e7ee56916785aa7c200aa00a859f8e5582460824b71', Func_ID=6640.0, Average='2829', Minimum='72.0', Maximum='5604.0'),
 Row(Func_index=662887, HashOwner='7432c37efe5a90415c2f0e67b3559899d15ec02766bb42d6e87e578baa49cc01', Func_ID=562.0, Average='4575', Minimum='4575.0', Maximum='4575.0'),
 Row(Func_index=662890, HashOwner='653935f489b6ad9cbfc2769a9838723a83f9a8505d6e323ffeb57eaa3fffd60b', Func_ID=4110.0, Average='2006', Minimum='2006.0', Maximum='2006.0'),
 Row(Func_index=662895, HashOwner='653935f489b6ad9cbfc2769a9838723a83f9a8505d6e323ffeb57eaa3fffd60b', Func_ID=2069.0, Average='84', Minimum='18.0', Maximum='168.0'),
 Row(Func_index=662896, HashOwner='d8a13b0cad5805789f66d4a2f0ec391b7e5d9bc72637c5ecba542d9ff31e7c7a', Func_ID=3592.0, Average='87', Minimum='0.0', Maximum='234.0'),
 Row(Func_index=662898, HashOwner='7369f94603b35208beb5add52dc9fcaa2198569e0a263072785fccd0fae537fe', Func_ID=1856.0, Average='46', Minimum='38.0', Maximum='53

In [8]:
## compare train and test set print out with the df asc and desc printouts above
## just double check the size again
print("Size of train set:",df_train.count())
print("Size of test set:",df_test.count())

StatementMeta(genterist, 2, 8, Finished, Available)

Size of train set: 136497
Size of test set: 500


# Export

In [9]:
## Export Train Set
df_train.coalesce(1).write.option("header","TRUE").csv("abfss://genfilesys@gensynapse.dfs.core.windows.net/httpTrigger_prediction/TCN_AzurePayload_TRAIN__v20230225")
## Export Test Set
df_test.coalesce(1).write.option("header","TRUE").csv("abfss://genfilesys@gensynapse.dfs.core.windows.net/httpTrigger_prediction/TCN_AzurePayload_TEST__v20230225")
print("Success!!")

StatementMeta(genterist, 2, 9, Finished, Available)

Success!!
